# Quantum Error Correction Review

## The basic problem

Quantum states are fragile:

- Qubits pick up random bit-flip errors: ∣0⟩↔∣1⟩

- Or phase-flip errors: ∣0⟩→∣0⟩,∣1⟩→−∣1⟩

- Or combinations (a Pauli 𝑋,𝑍,𝑌).

You cannot just “copy the qubit” like classical redundancy because of no-cloning. Instead:

  We encode one logical qubit into many physical qubits in a special entangled state, such that:

- We can measure error syndromes (what error happened)

- Without learning the logical state itself (so we don’t collapse it)

- Then correct it by conditional X/Z gates.

At high level, every stabilizer code consists of:

- Logical states:
∣0L⟩,∣1L⟩ (live in bigger Hilbert space)

- Stabilizers: commuting Pauli operators whose +1 eigenspace = code space

- Syndrome measurement: measure stabilizers → bit string → identify error

- Decoder: map syndrome → correction (X/Z/Y on certain qubits)

## Warm-up: 3-qubit repetition code (bit-flip)

Logical encoding:
- ∣0L⟩=∣000⟩
- ∣1L⟩=∣111⟩

So an arbitrary qubit
α∣0⟩+β∣1⟩ becomes

- α∣000⟩+β∣111⟩.

If one qubit flips (X error), e.g. on qubit 2:
- α∣010⟩+β∣101⟩

We can detect it by parity checks:

- Check qubit1 ⊕ qubit2

- Check qubit2 ⊕ qubit3

These are like measuring stabilizers:

- Z1Z2 and Z2Z3

The pattern of measurement outcomes (called syndrome) tells you which qubit flipped, then you apply X on that qubit to fix it.
- Syndrome tells : What error happened AND where it happened — without revealing the quantum state.

| Error case          | Z₁Z₂ | Z₂Z₃ | Syndrome |
| ------------------- | ---- | ---- | -------- |
| No error            | +1   | +1   | 00       |
| Bit-flip on qubit 1 | –1   | +1   | 10       |
| Bit-flip on qubit 2 | –1   | –1   | 11       |
| Bit-flip on qubit 3 | +1   | –1   | 01       |

Mapping:

00 → no error

10 → flip on qubit 1

11 → flip on qubit 2

01 → flip on qubit 3

This string (e.g., "10", "01") is the syndrome.

Then, the syndrome is the inputs to the decoder
- Decoder takes syndrome bits and outputs : "Apply this correction"

Example :

if syndrome == 10: apply X on qubit 0

if syndrome == 11: apply X on qubit 1

if syndrome == 01: apply X on qubit 2


## Experiment

1. Encode 1 logical qubit into 3 qubits

2. Add bit-flip noise

3. Measure syndromes (parity checks)

4. Correct based on syndromes

5. Compare with uncorrected case

In [2]:
!pip install qiskit-ibm-runtime
!pip install qiskit[visualization]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.4/377.4 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 108.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=1b484b2508cf638e68d1a8c515d3e95441c7efbc821dac127253b71762b47ff2
  Stored in directory: /root/.cache/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc


In [8]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

# 1. Create registers: 3 data qubits, 2 ancilla for syndrome, and classical bits
data = QuantumRegister(3, 'data')
anc = QuantumRegister(2, 'anc')
c_synd = ClassicalRegister(2, 'syndrome')
c_out = ClassicalRegister(1, 'out')
qc = QuantumCircuit(data, anc, c_synd, c_out) # the circuit contains data, anc, classical registers

# --- Step 1: Encode logical |+> = (|0> + |1>)/sqrt(2) as an example ---

# Prepare |+> on data[0]
qc.h(data[0]) # Hadamard

# Encode into repetition code: |+> -> (|000> + |111>)/sqrt(2)
qc.cx(data[0], data[1]) # CNOT
qc.cx(data[0], data[2])

# --- (later) we'll insert noise here ---

# --- Step 2: Syndrome measurement for bit-flip errors ---
# Measure Z1Z2 using anc[0]
qc.cx(data[0], anc[0]) # CNOT
qc.cx(data[1], anc[0])
qc.measure(anc[0], c_synd[0]) # Measure

# Measure Z2Z3 using anc[1]
qc.cx(data[1], anc[1])
qc.cx(data[2], anc[1])
qc.measure(anc[1], c_synd[1])

# --- Step 3: Decode back to one qubit AFTER correction (we'll add correction later) ---
# Majority vote in data qubits: here for simplicity, we'll just decode by:
# data[0] as representative, but in a more complete example you'd classically process syndrome
qc.barrier()
qc.cx(data[0], data[1])
qc.cx(data[0], data[2])
qc.measure(data[0], c_out[0])

print(qc)


            ┌───┐                                     ░           ┌─┐
    data_0: ┤ H ├──■────■────■────────────────────────░───■────■──┤M├
            └───┘┌─┴─┐  │    │                        ░ ┌─┴─┐  │  └╥┘
    data_1: ─────┤ X ├──┼────┼────■───────■───────────░─┤ X ├──┼───╫─
                 └───┘┌─┴─┐  │    │       │           ░ └───┘┌─┴─┐ ║ 
    data_2: ──────────┤ X ├──┼────┼───────┼────■──────░──────┤ X ├─╫─
                      └───┘┌─┴─┐┌─┴─┐┌─┐  │    │      ░      └───┘ ║ 
     anc_0: ───────────────┤ X ├┤ X ├┤M├──┼────┼──────░────────────╫─
                           └───┘└───┘└╥┘┌─┴─┐┌─┴─┐┌─┐ ░            ║ 
     anc_1: ──────────────────────────╫─┤ X ├┤ X ├┤M├─░────────────╫─
                                      ║ └───┘└───┘└╥┘ ░            ║ 
syndrome: 2/══════════════════════════╩════════════╩═══════════════╬═
                                      0            1               ║ 
     out: 1/═══════════════════════════════════════════════════════╩═
                    

That’s the “skeleton.” Now we add noise model and conditional correction.

### Adding bit-flip noise

We can simulate a bit-flip channel on each data qubit:

In [11]:
!pip install qiskit-aer
from qiskit_aer.noise import NoiseModel, pauli_error

# Bit-flip probability
p = 0.1
bit_flip = pauli_error([('X', p), ('I', 1-p)])

noise_model = NoiseModel()
for qubit in range(3):
    noise_model.add_all_qubit_quantum_error(bit_flip, ['id'])  # we'll use idle 'id' gates to inject noise

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 110.2 MB/s eta 0:00:00


In [12]:
# Then insert idle gates after encoding (so noise applies):
# Just after encoding:
qc.id(data[0])
qc.id(data[1])
qc.id(data[2])

### Syndromes → corrections

The “proper” way is:

- Run the circuit once on real hardware with mid-circuit measurements and if statements.

- On a simulator, we can also separate it into two circuits:

(1) run up to syndrome measurement,

(2) analyze counts and then build a second circuit that applies the best correction.

To keep it simpler in one shot, we can use classical conditions (Aer supports this):

In [23]:
from qiskit.circuit.library import XGate

# After measuring syndrome bits c_synd[0], c_synd[1] we can correct:

# Create a single XGate instruction instance
x_instruction = XGate()

# If syndrome == 01 -> error on qubit 1 (data[1])
# Create a mutable copy of the XGate and set its condition
conditioned_x_01 = x_instruction.to_mutable()
conditioned_x_01.condition = (c_synd, 0b01)
qc.append(conditioned_x_01, [data[1]])

# If syndrome == 10 -> error on qubit 0 (data[0])
conditioned_x_10 = x_instruction.to_mutable()
conditioned_x_10.condition = (c_synd, 0b10)
qc.append(conditioned_x_10, [data[0]])

# If syndrome == 11 -> error on qubit 2 (data[2])
conditioned_x_11 = x_instruction.to_mutable()
conditioned_x_11.condition = (c_synd, 0b11)
qc.append(conditioned_x_11, [data[2]])

In [25]:
from qiskit_aer import Aer # Import Aer from qiskit_aer

backend = Aer.get_backend('qasm_simulator')
job = backend.run(qc, shots=8192, noise_model=noise_model) # Use backend.run() instead of execute()
result = job.result()
counts = result.get_counts()
print(counts)

{'0 00': 4062, '1 00': 4130}


You’ll get probabilities of final measurement “0” vs “1” in c_out[0].
Your circuit had two separate classical registers:
- c_out : final logical measurement (1 bit)

- c_synd : syndrome bits (2 bits)

So:

"0 00" means:

- final logical output = 0

- syndrome bits = 00 (detected no error)

"1 00" means:

- final logical output = 1

- syndrome bits = 00  (detected no error)

And your counts:

- "0 00" → 4062 times

- "1 00" → 4130 times

Total ≈ 8192 shots (expected).